In [1]:
import rst_lib
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Retrieving data

Set `DATA_PATH` above the the path to `rst_discourse_treebank/` on your machine (after unzipping [`LDC2002T07.tgz`](https://catalog.ldc.upenn.edu/LDC2002T07))

In [2]:
DATA_PATH = "./rst_discourse_treebank/"
annotation_pairs = rst_lib.get_double_annotated_train_files(DATA_PATH, valid_only=True)




N
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
N
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
F

AssertionError: 

# Basic counts

We exclude the four shortest double-annotate documents for simplicity. They have very higgh agreement and are not very realistic documents anyway. Note that the prevalence of span errors does not seem to be correlated with document lenght.

In [ ]:
MIN_EDU_COUNT = 7

pair_counts = []
for pair in annotation_pairs:
  x = pair.agreement_scores
  num_edus = len(pair.gold_annotation.edus)
  if num_edus > MIN_EDU_COUNT:
    x.update({
      "identifier": pair.identifier,
      "num_edus": num_edus,
    })
    pair_counts.append(x)
df = pd.DataFrame.from_dict(pair_counts)

In [ ]:
sns.set(rc={'figure.figsize':(15,5)})
ax = sns.scatterplot(data=df, x='num_edus', y='S' )
sns.scatterplot(data=df, x='num_edus', y='N', ax=ax )
#sns.scatterplot(data=df, x='num_edus', y='S', ax=ax )
#sns.scatterplot(data=df, x='num_edus', y='F', ax=ax )
plt.xscale('log')

# Question: Is jitter ok if the axis is in log scale??

# Span height

In order to compare across documents with different lengths, we characterize each span in the tree with a measure of its normalized height.

$h_{N} = \dfrac{a}{a+b}$

Where $a$ is the span's distance from the root and $b$ is the length of the longest path from the span to any of its descendents.

We show that the distribution of ?disagreements over different span heights is bimodal -- disagreements tend to occur at the preterminal level and close to the root


Some options for where span errors can occur:
* Close to the root
* Close to the leaves
* ]

we look at trees of height at least 5
% errors in bottom two:
% errors in the top two:
% errors in the middle

^ this is ok because they ahve very similar amounts of nodes

In [ ]:
def mean(l):
  return sum(l)/len(l)

span_list = []
# List of dictionaries of the format {identifier: span: avg_depth: avg_height: status:}

for pair in annotation_pairs:
  main_spans = pair.gold_span_map
  double_spans = pair.predicted_span_map
  
  num_edus = mean([len(main_spans), len(double_spans)])
    
  for span in set(main_spans.keys()).union(double_spans.keys()):
    if tuple in [type(main_spans.get(span, None)),type(double_spans.get(span, None)) ]: # cleaner way to avoid 
      continue
    is_common = span in main_spans and span in double_spans
#     if is_common:
#       height = mean([main_spans[span]['height'], double_spans[span]['height']])
#       depth = mean([main_spans[span]['depth'], double_spans[span]['depth']])
#     else:
#       relevant_map, = [span_map for span_map in [main_spans, double_spans] if span in span_map]
#       height = relevant_map[span]['height']
#       depth = relevant_map[span]['depth']
      
      
#     if height < 3:
#       level = "leaf"
#     elif depth < 3:
#       level = "root"
#     else:
#       level = "middle"
      
      
    span_list.append({
        'identifier': pair.identifier,
        'span': span,
#         'avg_height': height,
#         'avg_depth': depth,
        'width': span[1] - span[0],
        #'status': 'common' if is_common else 'unique', 
        'status': 1 if is_common else 0, 
        'dummy':1,
        'total_edus': num_edus,
#         'level': level,
    }
                    )
    
df = pd.DataFrame.from_dict(span_list)

In [ ]:
df.groupby(['level'])['status'].mean()

# Rendering trees

This dumps phylogenetic tree-style renders of trees to `binary_parses/`

In [ ]:
for pair in sorted(annotation_pairs, key=lambda x:len(x.input_text))[:5]:  
  pair.gold_annotation.render(
        f"binary_parses/{pair.identifier}_gold.pdf")   
  pair.predicted_annotation.render(
        f"binary_parses/{pair.identifier}_predicted.pdf")

In [ ]:
import ete3
tree = ete3.Tree('((A,B), D);')

# --------- Data ---------------
#Long descriptions to attach to nodes
long_desc = {'A': '''This is a long, multi-line 
description of A''',
            'B': '''There is also a lot of potentially superfluous
information about B, 
how
many 
lines
can 
we 
possibly
add''',
            'C': '''There is so much information
about C that we might even need
3 lines to cover it all.''',
            'D': '''But D is pretty short'''}

#Define nodes that need certain labels attached
elaborations = ['A']
refutations = ['D']

# --------- Formatting -----------
#'elaboration', 'refutation' text faces
rst_faces = {'elaboration': ete3.faces.TextFace('elaboration',
                                                 fsize=6,
                                                 fgcolor='blue'),
            'refutation':   ete3.faces.TextFace('refutation',
                                                 fsize=6,
                                                 fgcolor='red')}


#Define the style for the vertical bar linking certain nodes
#an alternative to the "arrow" idea
elaboration_node_style = ete3.NodeStyle()
elaboration_node_style['vt_line_color'] = '#ff0000'
elaboration_node_style['vt_line_width'] = 8
elaboration_node_style['size'] = 0 #important to remove the little circle

#An "arrow" to put on a node - probably would want an ImgFace
direction_marker = ete3.faces.RectFace(5,5,'#0000ff', '#000000')

#Define a layout function to draw node stuff on the fly
def layout_fn(node):
    if node.is_leaf():
        desc_face = ete3.faces.TextFace(long_desc[node.name],
                                       fsize=8)
        ete3.faces.add_face_to_node(desc_face, node, column=0,
                                   aligned=True)
    if node.name in elaborations:
        ete3.faces.add_face_to_node(rst_faces['elaboration'],
                                    node,
                                    column=0,
                                   position='branch-top')
    if node.name in refutations:
        ete3.faces.add_face_to_node(rst_faces['refutation'],
                            node,
                            column=0,
                           position='branch-top')
    #"Arrows" graphics need to be drawn on parent nodes based on children
    if any([x.name in elaborations for x in node.children]):
        node.set_style(elaboration_node_style)
        ete3.faces.add_face_to_node(direction_marker, node,
                                   column=0,
                                   position='branch-bottom')
                                                     
#Make the tree ultrametric
tree.convert_to_ultrametric()
for node in tree.traverse():
    node.img_style['size'] = 0

#Define the TreeStyle object    
ts = ete3.TreeStyle()
#Change x-scale
ts.scale = 50
#Change y-scale
ts.branch_vertical_margin  = 10
#Draw tree right-to-left
ts.orientation = 1
#Remove the scale bar (and also branch lengths 
#which show up when scale bar is removed)
ts.show_scale = False
ts.show_branch_length = False

#Layout_fn is applied last after any changes made above
ts.layout_fn = layout_fn

tree.show(tree_style=ts)